## Statistics

In [1]:
# imports
import pandas as pd; pd.set_option('display.max_columns', None)
import scipy.stats
from tqdm.notebook import tqdm
import pingouin as pg

#### Phonetic Clustering Score

1-sample t-test against chance (H = 0, J = 0.5).  FDR correction (Benjamini-Hochberg) within experiment type.

In [2]:
def pcs_statistics(pcs_H_data_bsa, pcs_HS_data_bsa, pcs_HR_data_bsa, pcs_J_data_bsa, pcs_JFL_data_bsa):
    stats = []

    # H
    for et, data in pcs_H_data_bsa.groupby('exp_type'):
        res = scipy.stats.ttest_1samp(data.pcs, popmean=0, nan_policy='omit', alternative='two-sided')
        stats.append(('H', et, res.statistic, res.pvalue, res.df))
    
    # HS
    for et, data in pcs_HS_data_bsa.groupby('exp_type'):
        res = scipy.stats.ttest_1samp(data.pcs, popmean=0, nan_policy='omit', alternative='two-sided')
        stats.append(('HS', et, res.statistic, res.pvalue, res.df))
    
    # HR  
    for et, data in pcs_HR_data_bsa.groupby('exp_type'):
        res = scipy.stats.ttest_1samp(data.pcs, popmean=0, nan_policy='omit', alternative='two-sided')
        stats.append(('HR', et, res.statistic, res.pvalue, res.df))

    # J
    for et, data in pcs_J_data_bsa.groupby('exp_type'):
        res = scipy.stats.ttest_1samp(data.pcs, popmean=0.5, nan_policy='omit', alternative='two-sided')
        stats.append(('J', et, res.statistic, res.pvalue, res.df))
        
    # JFL
    for et, data in pcs_JFL_data_bsa.groupby('exp_type'):
        res = scipy.stats.ttest_1samp(data.pcs, popmean=0.5, nan_policy='omit', alternative='two-sided')
        stats.append(('JFL', et, res.statistic, res.pvalue, res.df))

    # save results as dataframe
    stats = pd.DataFrame(stats, columns=['version', 'exp_type', 't_stat', 'p_val', 'dof'])
    
    """
    # FDR correction (within experiment type)
    pcs_stats = []
    for et, stat in stats.groupby('exp_type'):
        stat['p_val_fdr'] = scipy.stats.false_discovery_control(stat.p_val, method='bh')
        pcs_stats.append(stat)
    
    return pd.concat(pcs_stats, ignore_index=True)
    """
    
    # FDR correction (across experiment type)
    stats['p_val_fdr'] = scipy.stats.false_discovery_control(stats.p_val, method='bh')
    
    return stats

In [3]:
pcs_H_data_bsa = pd.read_csv('analyses/dataframes/pcs_H_data_bsa.csv')
pcs_HS_data_bsa = pd.read_csv('analyses/dataframes/pcs_HS_data_bsa.csv')
pcs_HR_data_bsa = pd.read_csv('analyses/dataframes/pcs_HR_data_bsa.csv')
pcs_J_data_bsa = pd.read_csv('analyses/dataframes/pcs_J_data_bsa.csv')
pcs_JFL_data_bsa = pd.read_csv('analyses/dataframes/pcs_JFL_data_bsa.csv')

pcs_stats = pcs_statistics(pcs_H_data_bsa, pcs_HS_data_bsa, pcs_HR_data_bsa, pcs_J_data_bsa, pcs_JFL_data_bsa)
pcs_stats.to_csv('statistics/dataframes/pcs_stats.csv', index=False)
pcs_stats

,version,exp_type,t_stat,p_val,dof,p_val_fdr
0,H,intracranial,1.078819,2.812904e-01,418,4.688174e-01
1,H,scalp,7.438854,1.380333e-11,126,9.834633e-11
2,HS,intracranial,0.948109,3.436218e-01,418,4.908883e-01
3,HS,scalp,7.371387,1.966927e-11,126,9.834633e-11
4,HR,intracranial,0.068216,9.456466e-01,417,9.469702e-01
5,HR,scalp,3.181026,1.847871e-03,126,4.619678e-03
6,J,intracranial,-2.442989,1.497819e-02,419,2.995639e-02
7,J,scalp,0.103177,9.179862e-01,126,9.469702e-01
8,JFL,intracranial,0.066552,9.469702e-01,419,9.469702e-01
9,JFL,scalp,4.872419,3.245882e-06,126,1.081961e-05


#### Temporal and Semantic Clustering Scores

1-sample t-test against chance (0.5).

In [4]:
def cs_statistics(cs_data_bsa, cs):
    stats = []

    for et, data in cs_data_bsa.groupby('exp_type'):
        res = scipy.stats.ttest_1samp(data[cs], popmean=0.5, nan_policy='omit', alternative='two-sided')
        stats.append((et, res.statistic, res.pvalue, res.df))

    # save results as dataframe
    stats = pd.DataFrame(stats, columns=['exp_type', 't_stat', 'p_val', 'dof'])
    
    return stats

In [5]:
tcs_data_bsa = pd.read_csv('analyses/dataframes/tcs_data_bsa.csv')
tcs_stats = cs_statistics(tcs_data_bsa, 'tcs')
tcs_stats.to_csv('statistics/dataframes/tcs_stats.csv', index=False)
tcs_stats

,exp_type,t_stat,p_val,dof
0,intracranial,29.929751,4.234878e-106,419
1,scalp,28.948883,1.607748e-57,126


In [6]:
scs_data_bsa = pd.read_csv('analyses/dataframes/scs_data_bsa.csv')
scs_stats = cs_statistics(scs_data_bsa, 'scs')
scs_stats.to_csv('statistics/dataframes/scs_stats.csv', index=False)
scs_stats

,exp_type,t_stat,p_val,dof
0,intracranial,4.769839,2.549484e-06,419
1,scalp,24.917466,1.580761e-50,126


#### Phonetic Intrusions L

1-sample t-test against 0 with two-sided alternative hypothesis for the (PLI/ELI - PLI/ELI control).  Paired t-test comparing (PLI - PLI control & ELI - ELI control).  FDR correction (Benjamini-Hochberg) within experiment type.  

In [29]:
def psim_intr_l_statistics(psim_intr_l_H_data_bsa, psim_intr_l_J_data_bsa):
    stats = []

    # H
    for et, data in psim_intr_l_H_data_bsa.groupby('exp_type'):
        res_pli = scipy.stats.ttest_1samp(data.pli_delta, 0, nan_policy='omit', alternative='two-sided')
        res_eli = scipy.stats.ttest_1samp(data.eli_delta, 0, nan_policy='omit', alternative='two-sided')
        res_intr = scipy.stats.ttest_rel(data.eli_delta, data.pli_delta, nan_policy='omit', alternative='two-sided')

        stats.append(('H', et, 'pli', res_pli.statistic, res_pli.pvalue, res_pli.df))
        stats.append(('H', et, 'eli', res_eli.statistic, res_eli.pvalue, res_eli.df))
        stats.append(('H', et, 'intr', res_intr.statistic, res_intr.pvalue, res_intr.df))
        
    # J
    for et, data in psim_intr_l_J_data_bsa.groupby('exp_type'):
        res_pli = scipy.stats.ttest_1samp(data.pli_delta, 0, nan_policy='omit', alternative='two-sided')
        res_eli = scipy.stats.ttest_1samp(data.eli_delta, 0, nan_policy='omit', alternative='two-sided')
        res_intr = scipy.stats.ttest_rel(data.eli_delta, data.pli_delta, nan_policy='omit', alternative='two-sided')

        stats.append(('J', et, 'pli', res_pli.statistic, res_pli.pvalue, res_pli.df))
        stats.append(('J', et, 'eli', res_eli.statistic, res_eli.pvalue, res_eli.df))
        stats.append(('J', et, 'intr', res_intr.statistic, res_intr.pvalue, res_intr.df))

    # save results as dataframe
    stats = pd.DataFrame(stats, columns=['metric', 'exp_type', 'comparison', 't_stat', 'p_val', 'dof'])
    
    """
    # FDR correction (within experiment type)
    psim_intr_stats = []
    for et, stat in stats.groupby('exp_type'):
        stat['p_val_fdr'] = scipy.stats.false_discovery_control(stat.p_val, method='bh')
        psim_intr_stats.append(stat)
        
    return pd.concat(psim_intr_stats, ignore_index=True)
    """
    
    # FDR correction (across experiment type)
    stats['p_val_fdr'] = scipy.stats.false_discovery_control(stats.p_val, method='bh')
    
    return stats

In [31]:
psim_intr_l_H_data_bsa = pd.read_csv('analyses/dataframes/psim_intr_l_H_data_bsa.csv')
psim_intr_l_J_data_bsa = pd.read_csv('analyses/dataframes/psim_intr_l_J_data_bsa.csv')
psim_intr_l_stats = psim_intr_l_statistics(psim_intr_l_H_data_bsa, psim_intr_l_J_data_bsa)
psim_intr_l_stats.to_csv('statistics/dataframes/psim_intr_l_stats.csv', index=False)
psim_intr_l_stats

,metric,exp_type,comparison,t_stat,p_val,dof,p_val_fdr
0,H,intracranial,pli,7.058990,7.186247e-12,411,1.077937e-11
1,H,intracranial,eli,15.187511,1.303829e-41,408,1.564595e-40
2,H,intracranial,intr,6.707621,6.770853e-11,400,9.027804e-11
3,H,scalp,pli,8.610787,2.508825e-14,126,5.017650e-14
4,H,scalp,eli,12.502790,8.002168e-24,126,2.400650e-23
5,H,scalp,intr,4.968172,2.153732e-06,126,2.584478e-06
6,J,intracranial,pli,3.485083,5.449269e-04,411,5.449269e-04
7,J,intracranial,eli,14.787136,6.357048e-40,408,3.814229e-39
8,J,intracranial,intr,8.589359,1.976878e-16,400,4.744508e-16
9,J,scalp,pli,4.739637,5.686221e-06,126,6.203150e-06


In [12]:
# run anova on subject-averages (inbalance in data per subject)
def psim_intr_l_mm_anova(psim_intr_l_data_bsa):
    # re-organize dataframe
    dfm = pd.melt(psim_intr_l_data_bsa, id_vars=['subject', 'exp_type', 'experiment'],
                  value_vars=['pli_delta', 'eli_delta'], var_name='intr_type', value_name='delta')
    
    # only subjects with data in both conditions
    subs_balanced = []
    for (sub, et, exp), data in dfm.groupby(['subject', 'exp_type', 'experiment']):
        if all([x in data.intr_type.unique() for x in ['pli_delta', 'eli_delta']]):
            subs_balanced.append(sub)

    df = dfm[dfm.subject.isin(subs_balanced)]
    
    anova_results = pg.mixed_anova(data=df, dv='delta', within='intr_type', subject='subject', between='exp_type')
    return anova_results, df

In [13]:
# binary metric
psim_intr_l_H_data_bsa = pd.read_csv('analyses/dataframes/psim_intr_l_H_data_bsa.csv')
res_anova_l_H, df = psim_intr_l_mm_anova(psim_intr_l_H_data_bsa)
res_anova_l_H

,Source,SS,DF1,DF2,MS,F,p-unc,np2,eps
0,exp_type,0.034270,1,526,0.034270,16.762932,4.901759e-05,0.030884,NaN
1,intr_type,0.092916,1,526,0.092916,54.668876,5.659706e-13,0.094148,1.0
2,Interaction,0.009432,1,526,0.009432,5.549724,1.884965e-02,0.010441,NaN


In [14]:
res_tukey_l_H = pg.pairwise_tukey(data=df, dv='delta', between='intr_type')
res_tukey_l_H

,A,B,mean(A),mean(B),diff,se,T,p-tukey,hedges
0,eli_delta,pli_delta,0.034479,0.015073,0.019406,0.002737,7.089643,1.946776e-12,0.432163


In [15]:
# Jaccard index
psim_intr_l_J_data_bsa = pd.read_csv('analyses/dataframes/psim_intr_l_J_data_bsa.csv')
res_anova_l_J, df = psim_intr_l_mm_anova(psim_intr_l_J_data_bsa)
res_anova_l_J

,Source,SS,DF1,DF2,MS,F,p-unc,np2,eps
0,exp_type,0.004047,1,526,0.004047,15.246093,1.066559e-04,0.028169,NaN
1,intr_type,0.025344,1,526,0.025344,91.897081,3.620477e-20,0.148726,1.0
2,Interaction,0.002255,1,526,0.002255,8.176704,4.411649e-03,0.015307,NaN


In [16]:
res_tukey_l_J = pg.pairwise_tukey(data=df, dv='delta', between='intr_type')
res_tukey_l_J

,A,B,mean(A),mean(B),diff,se,T,p-tukey,hedges
0,eli_delta,pli_delta,0.012666,0.002796,0.009869,0.001028,9.600604,0.0,0.585224


#### Phonetic Intrusions R

Paired t-tests (PLI-CR, ELI-CR, PLI-ELI).  FDR correction (Benjamnini-Hochberg) within experiment type

Repeated measures anova within experiment type.

In [35]:
def psim_intr_r_statistics(psim_intr_r_H_data_bsa, psim_intr_r_J_data_bsa):
    # re-organize dataframes
    dfH = psim_intr_r_H_data_bsa.pivot(index=['subject', 'exp_type', 'experiment'], columns='resp_type', values='psim')
    dfH.columns = [f"{col}_psim" for col in dfH.columns]
    dfH.reset_index(inplace=True)
    
    dfJ = psim_intr_r_J_data_bsa.pivot(index=['subject', 'exp_type', 'experiment'], columns='resp_type', values='psim')
    dfJ.columns = [f"{col}_psim" for col in dfJ.columns]
    dfJ.reset_index(inplace=True)
    
    stats = []
    
    # H
    for et, data in dfH.groupby('exp_type'):
        res_pli = scipy.stats.ttest_rel(data.pli_psim, data.cr_psim, nan_policy='omit', alternative='two-sided')
        res_eli = scipy.stats.ttest_rel(data.eli_psim, data.cr_psim, nan_policy='omit', alternative='two-sided')
        res_intr = scipy.stats.ttest_rel(data.eli_psim, data.pli_psim, nan_policy='omit', alternative='two-sided')

        stats.append(('H', et, 'pli', res_pli.statistic, res_pli.pvalue, res_pli.df))
        stats.append(('H', et, 'eli', res_eli.statistic, res_eli.pvalue, res_eli.df))
        stats.append(('H', et, 'intr', res_intr.statistic, res_intr.pvalue, res_intr.df))
        
    # J
    for et, data in dfJ.groupby('exp_type'):
        res_pli = scipy.stats.ttest_rel(data.pli_psim, data.cr_psim, nan_policy='omit', alternative='two-sided')
        res_eli = scipy.stats.ttest_rel(data.eli_psim, data.cr_psim, nan_policy='omit', alternative='two-sided')
        res_intr = scipy.stats.ttest_rel(data.eli_psim, data.pli_psim, nan_policy='omit', alternative='two-sided')

        stats.append(('J', et, 'pli', res_pli.statistic, res_pli.pvalue, res_pli.df))
        stats.append(('J', et, 'eli', res_eli.statistic, res_eli.pvalue, res_eli.df))
        stats.append(('J', et, 'intr', res_intr.statistic, res_intr.pvalue, res_intr.df))

    # save results as dataframe
    stats = pd.DataFrame(stats, columns=['metric', 'exp_type', 'comparison', 't_stat', 'p_val', 'dof'])
    
    """
    # FDR correction (within experiment type)
    psim_intr_stats = []
    for et, stat in stats.groupby('exp_type'):
        stat['p_val_fdr'] = scipy.stats.false_discovery_control(stat.p_val, method='bh')
        psim_intr_stats.append(stat)
        
    return pd.concat(psim_intr_stats, ignore_index=True)
    """

    # FDR correction (across experiment type)
    stats['p_val_fdr'] = scipy.stats.false_discovery_control(stats.p_val, method='bh')
    
    return stats

In [39]:
psim_intr_r_H_data_bsa = pd.read_csv('analyses/dataframes/psim_intr_r_H_data_bsa.csv')
psim_intr_r_J_data_bsa = pd.read_csv('analyses/dataframes/psim_intr_r_J_data_bsa.csv')
psim_intr_r_stats = psim_intr_r_statistics(psim_intr_r_H_data_bsa, psim_intr_r_J_data_bsa)
psim_intr_r_stats.to_csv('statistics/dataframes/psim_intr_r_stats.csv', index=False)
psim_intr_r_stats

,metric,exp_type,comparison,t_stat,p_val,dof,p_val_fdr
0,H,intracranial,pli,2.391236,0.017250,403,0.039062
1,H,intracranial,eli,3.194399,0.001513,398,0.009077
2,H,intracranial,intr,0.530420,0.596125,387,0.705437
3,H,scalp,pli,0.122623,0.902601,126,0.902601
4,H,scalp,eli,2.371971,0.019207,126,0.039062
5,H,scalp,intr,1.603628,0.111300,126,0.166950
6,J,intracranial,pli,1.277978,0.201993,403,0.269323
7,J,intracranial,eli,4.294789,0.000022,398,0.000264
8,J,intracranial,intr,2.674926,0.007792,387,0.031167
9,J,scalp,pli,-0.459523,0.646650,126,0.705437


In [ ]:
# run anova on subject-averages (inbalance in data per subject)
def psim_intr_r_mm_anova(psim_intr_r_data_bsa):
    # only subjects with data in all 3 conditions
    subs_balanced = []
    for (sub, et, exp), data in psim_intr_r_data_bsa.groupby(['subject', 'exp_type', 'experiment']):
        if all([x in data.resp_type.unique() for x in ['cr', 'pli', 'eli']]):
            subs_balanced.append(sub)

    df = psim_intr_r_data_bsa[psim_intr_r_data_bsa.subject.isin(subs_balanced)].query("resp_type != 'control'")
    
    anova_results = pg.mixed_anova(data=df, dv='delta', within='resp_type', subject='subject', between='exp_type')
    return anova_results, df

In [21]:
# binary metric
psim_intr_r_H_data_bsa = pd.read_csv('analyses/dataframes/psim_intr_r_H_data_bsa.csv')
res_anova_r_H, df = psim_intr_r_mm_anova(psim_intr_r_H_data_bsa)
res_anova_r_H

,Source,SS,DF1,DF2,MS,F,p-unc,p-GG-corr,np2,eps,sphericity,W-spher,p-spher
0,exp_type,0.026114,1,513,0.026114,1.226075,0.268691,NaN,0.002384,NaN,NaN,NaN,NaN
1,resp_type,0.200839,2,1026,0.100419,4.934290,0.007367,0.011308,0.009527,0.832748,False,0.799156,1.059327e-25
2,Interaction,0.025922,2,1026,0.012961,0.636852,0.529164,NaN,0.001240,NaN,NaN,NaN,NaN


In [22]:
res_tukey_r_H = pg.pairwise_tukey(data=df, dv='delta', between='resp_type')
res_tukey_r_H

,A,B,mean(A),mean(B),diff,se,T,p-tukey,hedges
0,cr,eli,0.005062,0.032526,-0.027463,0.008957,-3.066005,0.006245,-0.226800
1,cr,pli,0.005062,0.023186,-0.018123,0.008957,-2.023251,0.106974,-0.133884
2,eli,pli,0.032526,0.023186,0.009340,0.008957,1.042754,0.549972,0.054768


In [23]:
# Jaccard index
psim_intr_r_J_data_bsa = pd.read_csv('analyses/dataframes/psim_intr_r_J_data_bsa.csv')
res_anova_r_J, df = psim_intr_r_mm_anova(psim_intr_r_J_data_bsa)
res_anova_r_J

,Source,SS,DF1,DF2,MS,F,p-unc,p-GG-corr,np2,eps,sphericity,W-spher,p-spher
0,exp_type,0.003939,1,513,0.003939,1.328512,0.249607,NaN,0.002583,NaN,NaN,NaN,NaN
1,resp_type,0.058477,2,1026,0.029238,10.246763,0.000039,0.000117,0.019583,0.847689,False,0.820323,8.651072e-23
2,Interaction,0.006419,2,1026,0.003210,1.124825,0.325109,NaN,0.002188,NaN,NaN,NaN,NaN


In [24]:
res_tukey_r_J = pg.pairwise_tukey(data=df, dv='delta', between='resp_type')
res_tukey_r_J

,A,B,mean(A),mean(B),diff,se,T,p-tukey,hedges
0,cr,eli,0.000450,0.014491,-0.014041,0.003351,-4.189996,0.000087,-0.288790
1,cr,pli,0.000450,0.002732,-0.002281,0.003351,-0.680767,0.774747,-0.049025
2,eli,pli,0.014491,0.002732,0.011760,0.003351,3.509229,0.001342,0.182362


#### Correlations with Recall Probability

1-sample t-test against no correlation (0).  FDR correction (Benjamini-Hochberg) within experiment type.

In [25]:
def p_recall_corrs_statistics(p_recall_corrs_bsa):
    stats = []
    for et, data in p_recall_corrs_bsa.groupby('exp_type'):
        if len(data) > 1:
            res_pcs_H = scipy.stats.ttest_1samp(data.r_pcs_H, popmean=0, nan_policy='omit', alternative='two-sided')
            res_pcs_J = scipy.stats.ttest_1samp(data.r_pcs_J, popmean=0, nan_policy='omit', alternative='two-sided')
            res_tcs = scipy.stats.ttest_1samp(data.r_tcs, popmean=0, nan_policy='omit', alternative='two-sided')
            res_scs = scipy.stats.ttest_1samp(data.r_scs, popmean=0, nan_policy='omit', alternative='two-sided')

            stats.append(('pcs_H', et, res_pcs_H.statistic, res_pcs_H.pvalue, res_pcs_H.df))
            stats.append(('pcs_J', et, res_pcs_J.statistic, res_pcs_J.pvalue, res_pcs_J.df))
            stats.append(('tcs', et, res_tcs.statistic, res_tcs.pvalue, res_tcs.df))
            stats.append(('scs', et, res_scs.statistic, res_scs.pvalue, res_scs.df))

    stats = pd.DataFrame(stats, columns=['label', 'exp_type', 't_stat', 'p_val', 'dof'])
    
    """
    # FDR correction (within experiment type)
    p_recall_corrs_stats = []
    for et, stat in stats.groupby('exp_type'):
        stat['p_val_fdr'] = scipy.stats.false_discovery_control(stat.p_val, method='bh')
        p_recall_corrs_stats.append(stat)

    return pd.concat(p_recall_corrs_stats, ignore_index=True)
    """
    
    # FDR correct (across experiment type)
    stats['p_val_fdr'] = scipy.stats.false_discovery_control(stats.p_val, method='bh')
    
    return stats

In [26]:
p_recall_corrs_bsa = pd.read_csv('analyses/dataframes/p_recall_corrs_bsa.csv')
p_recall_corrs_stats = p_recall_corrs_statistics(p_recall_corrs_bsa)
p_recall_corrs_stats.to_csv('statistics/dataframes/p_recall_corrs_stats.csv', index=False)
p_recall_corrs_stats

,label,exp_type,t_stat,p_val,dof,p_val_fdr
0,pcs_H,intracranial,3.274474,1.147997e-03,412,0.003061
1,pcs_J,intracranial,2.202353,2.819365e-02,412,0.037592
2,tcs,intracranial,-0.510788,6.097730e-01,412,0.609773
3,scs,intracranial,-1.648678,9.997591e-02,412,0.114258
4,pcs_H,scalp,2.459068,1.528696e-02,126,0.024459
5,pcs_J,scalp,3.174009,1.889701e-03,126,0.003779
6,tcs,scalp,5.448460,2.564687e-07,126,0.000002
7,scs,scalp,3.905863,1.522571e-04,126,0.000609
